In [7]:
#Steps for today:
#Import the data (V&J)
#Exploring the features (V&J)
#Clean and normalize the data (V&J)
#Decide on a classification algorithm (V&J)
#First trainings (V&J)
#Calculate the scoring and tweak/consolidate features accordingly (V&J)
#First submission
#!pip3 install markupsafe==2.0.1
#!conda install -y -c conda-forge pandas-profiling
#!pip install pandas-profiling
#!pip install category-encoders
#!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.1 MB/s eta 0:00:00a 0:00:01m


In [36]:
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import category_encoders as ce
from pathlib import Path
import matplotlib.pyplot as plt

In [37]:
train_values = pd.read_csv('train_values.csv', sep=',').drop(columns=["legal_ownership_status","count_families","has_secondary_use","has_secondary_use_agriculture","has_secondary_use_hotel","has_secondary_use_rental","has_secondary_use_institution","has_secondary_use_school","has_secondary_use_industry","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])
train_labels = pd.read_csv('train_labels.csv', sep=',')
x_test = pd.read_csv('test_values.csv', sep=',').drop(columns=["legal_ownership_status","count_families","has_secondary_use","has_secondary_use_agriculture","has_secondary_use_hotel","has_secondary_use_rental","has_secondary_use_institution","has_secondary_use_school","has_secondary_use_industry","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])
#We only use the first 10000 data points
prof = ProfileReport(train_values)
train_values

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,n,f,j,s,q
260597,669485,17,715,2060,2,0,6,5,t,r,n,f,q,s,d
260598,602512,17,51,8163,3,55,6,7,t,r,q,f,q,s,d
260599,151409,26,39,1851,2,10,14,6,t,r,x,v,s,j,d


In [41]:
#Encode

ce_te = ce.TargetEncoder(cols=["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration"])
    
adapted_train_labels = train_labels.drop(columns=['building_id']).values.flatten()

def encode_values(normalized_train_values,X,y):
    #create an object of the Targetencoder
    ce_te.fit(X,y)
    encoded_columns = ce_te.transform(X)
    # selecting old value
    normalized_train_values[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration"]] = encoded_columns[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration"]]
    new_normalized_train_values= normalized_train_values.drop(columns=['building_id'])

    return new_normalized_train_values

    
X = train_values[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration"]]
y = train_values['building_id']    
train_values_normalized_encoded = encode_values(train_values.copy(),X,y)

X_test = x_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration"]]
y_test = x_test['building_id']

test_values_normalized_encoded = encode_values(x_test.copy(),X_test,y_test)



'X_test = x_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]]\ny_test = x_test[\'building_id\']\n\n#create an object of the Targetencoder\nce_te.fit(X_test,y_test)\nencoded_columns_test = ce_te.transform(X_test)\n#normalized_train_values_test = x_test.copy()\n\n# selecting old value\nnormalized_train_values_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]] = encoded_columns_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]]\nnew_normalized_train_values_test= normalized_train_values_test.drop(columns=[\'building_id\'])\nnew_normalized_train_values_test\n\nencode_values(x_test.copy())'

In [42]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier()
clf = clf.fit(train_values_normalized_encoded,adapted_train_labels)
y_pred=clf.predict(test_values_normalized_encoded)
y_pred

array([3, 2, 2, ..., 2, 2, 2])

In [43]:
from sklearn.model_selection import cross_val_score

#10-Fold Cross validation
print(np.mean(cross_val_score(clf, train_values_normalized_encoded, adapted_train_labels)))

0.7114707894639698


In [100]:
#Get submission ready for random forest attempt
import base64

submission_random_forest = x_test["building_id"]
i = pd.DataFrame(np.array(y_pred))

submission_concatenated = pd.concat([submission_random_forest, i], ignore_index="true", axis=1)
submission_renamed_columns = submission_concatenated.rename(columns={0: "building_id", 1: "damage_grade"})
filepath = Path('./submission_random_forest_1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
submission_renamed_columns.to_csv(filepath, index=False)
